In [1]:
RAWTEXT = "../input/cmu-us-awb-arctic-tts-dataset/cmu_us_awb_arctic/etc/txt.done.data"

In [33]:
NORMS = {
    "0.75": "zero point seven five",
    "t.h": "t h",
    "1880": "eighteen eighty",
    "16": "sixteenth",
    "1908": "nineteen oh eight",
    "18": "eighteenth",
    "17": "seventeenth",
    "29th": "twenty ninth",
    "mrs": "misses",
    "etc": "etcetera",
    "etc.": "etcetera",
    "to-day": "today",
    "to-day's": "today's",
    "to-morrow": "tomorrow"
}

In [15]:
def _check_apos(word):
    if word.endswith("'s"):
        return word
    elif word.endswith("s'"):
        return word
    elif word.endswith("'d"):
        return word
    elif word.endswith("'ve"):
        return word
    elif word.endswith("'re"):
        return word
    elif word.endswith("'ll"):
        return word
    elif word.endswith("n't"):
        return word
    elif word.endswith("'ve"):
        return word
    elif word in ["i'm", "'em", "o'brien"]:
        return word
    else:
        return word.replace("'", "")

def fix_apos(text):
    words = [_check_apos(w) for w in text.split(" ")]
    return " ".join(words)

In [27]:
def normalise(text):
    if text[-1] == ".":
        text = text[:-1]
    text = text.lower()
    words = []
    text = text.replace(",", "")
    for word in text.split(" "):
        if word in NORMS:
            words.append(NORMS[word])
        else:
            words.append(word)
    text = " ".join(words)
    text = text.replace(".", "")
    text = text.replace("?", "")
    text = text.replace("!", "")
    text = text.replace(":", "")
    text = text.replace(";", "")
    text = text.replace("--", " ")
    text = text.replace("  ", " ")
    text = text.replace(" - ", " ")
    text = text.replace("to- morrow", "tomorrow")
    text = fix_apos(text)
    text = text.replace("-", " ")
    return text.strip().upper()

In [28]:
data = {}
with open(RAWTEXT) as inf:
    for line in inf.readlines():
        first_space = line.find(' ')
        first_quote = line.find('"')
        last_quote = line.rfind('"')
        id = line[first_space+1:first_quote].strip()
        text = line[first_quote+1:last_quote]
        #data[id] = text
        data[id] = normalise(text)

In [25]:
for datum in data.keys():
    if "'" in data[datum]:
        print(data[datum])
        print(fix_apos(data[datum]))
        print("\n")

In [29]:
ok_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ '"
for datum in data.keys():
    for char in data[datum]:
        if char not in ok_chars:
            print(datum, data[datum], char)

In [35]:
!cat ../input/cmu-us-awb-arctic-tts-dataset/cmu_us_awb_arctic/etc/txt.done.data |grep '\. '

In [32]:
!grep Dr ../input/cmu-us-awb-arctic-tts-dataset/cmu_us_awb_arctic/etc/txt.done.data

In [31]:
import IPython

def play_id(id):
    return IPython.display.Audio(f"../input/cmu-us-awb-arctic-tts-dataset/cmu_us_awb_arctic/wav/{id}.wav")

In [32]:
play_id("arctic_b0311")

In [13]:
!ls ../input/cmu-us-awb-arctic-tts-dataset/cmu_us_awb_arctic/wav/
